In [23]:
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv
from dotenv import load_dotenv
import os 
from pymongo import MongoClient

In [6]:
token = getpass()

In [ ]:
# I am going to request how many of the following locations I have in sydney, and where is located.

# Residential areas
# Parks for outdoor activities
# Universities 
# Gyms for fitness enthusiasts
# Social Clubs 
# Bars for post-game meals and socializing
# Public transportation options
# physiotherapy 
# Tennis store
# Parking for cars.

In [52]:
# I create a dictionary with all the locations and its code associated in 4sq.
dict_of_places = {'bar':13003,
                   'physiotherapy':15026,
                   'social_club':12097,
                   'tennis_store':17130,
                   'residential':12094,
                   'park': 16032,
                   'university': 12013,
                   'gym':18021,
                   'parking':19020,
                   'public_transport':19054,
                   'tennis_court':18047
                    }



In [48]:
# I create two functions, one to request to 4sq and the other one to insert the result to mongo.

def request_4sq (key,dictionary):
    ''' This functions takes the key and the dictionary
        returns the respons of the request to 4sq based on the key we passed. '''

    center_point = '-33.8149474%2C150.9999835'
    radius = 30000
    headers = {"accept": "application/json",
                    "Authorization": token }
    category_code = dictionary[key]
    limit = 30
    url = f"https://api.foursquare.com/v3/places/search?ll={center_point}&radius={radius}&categories={category_code}&fields=name%2Cgeocodes&limit={limit}"

    response = requests.get(url, headers=headers).json()
    return response['results']

def send_to_mongo (key,response):
    '''This function takes the key and the response from 4sq
        insert the result in mongo and returns a message confirming.'''
    client = MongoClient("localhost:27017")
    db = client['geospatial']
    collection = db[key]
    collection.insert_many(response)

In [51]:
# I loop through the dictionary of categories and codes, and I apply the two functions I created.
for key in dict_of_places.keys():
    response = (request_4sq(key,dict_of_places))
    send_to_mongo(key,response)
